In [1]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from simplet5 import *
import os
from utils import *

ddir = '../../data/'
dname = 'syntax-complete'
nbname = 't5-t1'
outputdir="/scratch/userid/" + nbname + '/'
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
resultsfile = 'results/' + nbname + '.csv'


Global seed set to 42


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2


env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
df_train = pd.read_csv(ddir + dname + '_train.csv')
df_test = pd.read_csv(ddir + dname + '_test.csv')
df_val = pd.read_csv(ddir + dname + '_val.csv')

In [4]:
def preprocess_df(df):
    X = []
    y = []
    for index, row in df.iterrows():
        try:
            tmp = ""
            tmp = "<claim> " + row['claim'] + "\n"
            tmp += "<context> " + row['context'] + "\n"
            tmp += "<source> " + row['Source'] + '\n'
            tmp += "<relation> " + row['Relation'] + '\n'
            tmp += "<target> " + row['Target'] + '\n'
            tmp2 = "<mismatch> " + row['Mismatch'] + '\n'
            X.append(tmp)
            y.append(tmp2)
        except:
            continue
    return X,y
def preprocess_df_r(df):
    X = []
    y = []
    for index, row in df.iterrows():
        # try:
        tmp = ""
        tmp = "<claim> " + row['claim'] + "\n" if isinstance(row['claim'], str) else "<claim> "
        tmp2 = "<target> " + row['Target'] + '\n' if isinstance(row['Target'], str) else "<target> "
        X.append(tmp)
        y.append(tmp2)
        # except:
        #     print(index, row)
        #     continue
    return X,y

In [5]:
X_train, y_train = preprocess_df_r(df_train)
X_val, y_val = preprocess_df_r(df_val)
X_test, y_test = preprocess_df_r(df_test)

In [6]:
len(X_train), len(X_val), len(X_test)

(8794, 1088, 1098)

In [7]:
train_df = pd.DataFrame({'source_text': X_train, 'target_text': y_train})    
val_df = pd.DataFrame({'source_text': X_val, 'target_text': y_val})    
test_df = pd.DataFrame({'source_text': X_test, 'target_text': y_test})   

In [8]:
train_df

source_text  \
0     <claim> Batman: The Killing Joke was illustrat...   
1     <claim> Cancer is a group of diseases involvin...   
2       <claim> Steffi Graf won 6 consecutive majors.\n   
3             <claim> Page Hamilton is only a singer.\n   
4     <claim> There are zero water in vapor and clou...   
...                                                 ...   
8789                 <claim> Basketball is a musical.\n   
8790  <claim> Nigeria is not one of the countries in...   
8791  <claim> Nikola Tesla refused to be a physicist.\n   
8792  <claim> Lion was not based on a book written b...   
8793  <claim> Jeff Bezos is the fifth-richest person...   

                                            target_text  
0     <target> by a British comics artist born in 19...  
1     <target> a group of diseases involving abnorma...  
2                       <target> 6 consecutive majors\n  
3                                   <target> a singer\n  
4                        <target> in vapor and clouds\n  
...                                                 ...  
8789                               <target> a musical\n  
8790  <target> one of the countries in the Next Elev...  
8791                             <target> a physicist\n  
8792        <target> a book written by Larry Buttrose\n  
8793   <target> the fifth-richest person in the world\n  

[8794 rows x 2 columns]

In [9]:
# load model
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

In [10]:
# train model
model.train(train_df=train_df,
            eval_df=val_df, 
            source_max_token_len=128, 
            target_max_token_len=128, 
            batch_size=16, 
            max_epochs=4, 
            outputdir = outputdir,
            )
            


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/userid/anaconda/envs/p3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in dirs:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result
find('simplet5-epoch-3*', outputdir)[0]

'/scratch/userid/t5-t1/simplet5-epoch-3-train-loss-0.0718-val-loss-0.0919'

In [12]:
from tqdm import tqdm
# load the best model

last_epoch_model = outputdir + find('simplet5-epoch-3*', outputdir)[0] # put the name here
model.load_model("t5-small", last_epoch_model, use_gpu=True)
# model.eval()
# for each test data perform prediction
predictions = []
for index, row in tqdm(test_df.iterrows()):
    prediction = model.predict(row['source_text'])[0]
    predictions.append(prediction)


# computer performance
test_df['predicted_label'] = predictions
test_df['original_label'] = test_df['target_text']

1098it [01:49, 10.01it/s]


In [13]:
df_test['predicted_label'] = predictions
df_test['original_label'] = test_df['target_text']

In [14]:
test_df

source_text  \
0                   <claim> Janet Varney is a German.\n   
1     <claim> Streptococcus is a genus of Gram-posit...   
2     <claim> Prometheus is a 2014 film that serves ...   
3       <claim> The Fosters is not a television show.\n   
4     <claim> Kajal Aggarwal has yet to star in any ...   
...                                                 ...   
1093  <claim> Evil Queen (Disney) was animated by Wa...   
1094   <claim> The Incredible Hulk is produced by DC.\n   
1095        <claim> Tim Rice wrote a 1917 rock opera.\n   
1096  <claim> The Prince of Egypt is an animated tel...   
1097      <claim> Scarlett Johansson has never acted.\n   

                                     target_text  \
0                            <target> a German\n   
1     <target> a genus of Gram-positive humans\n   
2                         <target> a 2014 film\n   
3                   <target> a television show\n   
4                        <target> in any films\n   
...                                          ...   
1093                   <target> by Walt Disney\n   
1094                            <target> by DC\n   
1095                <target> a 1917 rock opera\n   
1096      <target> an animated television show\n   
1097                            <target> acted\n   

                                   predicted_label  \
0                                         A German   
1                 target> a genus of Gram-positive   
2     target> a 2014 film that serves as a prequel   
3                        target> a television show   
4                             target> in any films   
...                                            ...   
1093                        target> by Walt Disney   
1094                                 target> by DC   
1095                     target> a 1917 rock opera   
1096           target> an animated television show   
1097                                 target> acted   

                                  original_label  
0                            <target> a German\n  
1     <target> a genus of Gram-positive humans\n  
2                         <target> a 2014 film\n  
3                   <target> a television show\n  
4                        <target> in any films\n  
...                                          ...  
1093                   <target> by Walt Disney\n  
1094                            <target> by DC\n  
1095                <target> a 1917 rock opera\n  
1096      <target> an animated television show\n  
1097                            <target> acted\n  

[1098 rows x 4 columns]

In [25]:
def process_output_r(x):
    x_split = x.split(' ')
    # ls = ['', '', '', '']
    ls = ['']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        if i == 'target>':
            itr = 0
        # elif i == 'mismatch>':
        #     itr = 3
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0]])


def process_input_r(x):
    x_split = x.split()
    # ls = ['', '', '', '']
    ls = ['']
    itr = None
    for i in x_split:
        # if i == 'source>':
        #     itr = 0
        # elif i == 'relation>':
        #     itr = 1
        if i == '<target>':
            itr = 0
        # elif i == 'mismatch>':
        #     itr = 3
        elif itr is not None:
            ls[itr] += i + ' '
    return pd.Series([ls[0]])

In [26]:
test_df[['pTarget']] = test_df['predicted_label'].apply(process_output_r)
test_df[['Target']] = test_df['original_label'].apply(process_input_r)

In [27]:
df_test['predicted_label'] = predictions
df_test['original_label'] = test_df['target_text']

In [28]:
test_df

source_text  \
0                   <claim> Janet Varney is a German.\n   
1     <claim> Streptococcus is a genus of Gram-posit...   
2     <claim> Prometheus is a 2014 film that serves ...   
3       <claim> The Fosters is not a television show.\n   
4     <claim> Kajal Aggarwal has yet to star in any ...   
...                                                 ...   
1093  <claim> Evil Queen (Disney) was animated by Wa...   
1094   <claim> The Incredible Hulk is produced by DC.\n   
1095        <claim> Tim Rice wrote a 1917 rock opera.\n   
1096  <claim> The Prince of Egypt is an animated tel...   
1097      <claim> Scarlett Johansson has never acted.\n   

                                     target_text  \
0                            <target> a German\n   
1     <target> a genus of Gram-positive humans\n   
2                         <target> a 2014 film\n   
3                   <target> a television show\n   
4                        <target> in any films\n   
...                                          ...   
1093                   <target> by Walt Disney\n   
1094                            <target> by DC\n   
1095                <target> a 1917 rock opera\n   
1096      <target> an animated television show\n   
1097                            <target> acted\n   

                                   predicted_label  \
0                                         A German   
1                 target> a genus of Gram-positive   
2     target> a 2014 film that serves as a prequel   
3                        target> a television show   
4                             target> in any films   
...                                            ...   
1093                        target> by Walt Disney   
1094                                 target> by DC   
1095                     target> a 1917 rock opera   
1096           target> an animated television show   
1097                                 target> acted   

                                  original_label  \
0                            <target> a German\n   
1     <target> a genus of Gram-positive humans\n   
2                         <target> a 2014 film\n   
3                   <target> a television show\n   
4                        <target> in any films\n   
...                                          ...   
1093                   <target> by Walt Disney\n   
1094                            <target> by DC\n   
1095                <target> a 1917 rock opera\n   
1096      <target> an animated television show\n   
1097                            <target> acted\n   

                                    pTarget                            Target  
0                                                                   a German   
1                 a genus of Gram-positive   a genus of Gram-positive humans   
2     a 2014 film that serves as a prequel                       a 2014 film   
3                        a television show                 a television show   
4                             in any films                      in any films   
...                                     ...                               ...  
1093                        by Walt Disney                    by Walt Disney   
1094                                 by DC                             by DC   
1095                     a 1917 rock opera                 a 1917 rock opera   
1096           an animated television show       an animated television show   
1097                                 acted                             acted   

[1098 rows x 6 columns]

In [29]:
def exact_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        if orig[i].strip() == predicted[i].strip():
            match += 1
    return match/len(orig)

def partial_match(orig, predicted):
    match = 0
    for i in range(len(orig)):
        orig_split = orig[i].split()
        predicted_split = predicted[i].split()
        if len(orig_split) == 0:
            if len(predicted_split) == 0:
                match += 1
            else:
                match += 0
            continue
        ints = list(set(orig_split) & set(predicted_split))
        match += len(ints)/len(orig_split)
    return match/len(orig)

In [30]:
# exact_match(list(test_df.pSource), list(test_df.Source)), partial_match(list(test_df.pSource), list(test_df.Source))

In [31]:
# exact_match(list(test_df.pRelation), list(test_df.Relation)), partial_match(list(test_df.pRelation), list(test_df.Relation))

In [32]:
exact_match(list(test_df.pTarget), list(test_df.Target)), partial_match(list(test_df.pTarget), list(test_df.Target))

(0.6857923497267759, 0.88637225653691)

In [23]:
test_df.to_csv(resultsfile)